# O quão bom é Max Verstappen? Uma análise de dados

Neste notebook, apresento uma análise de dados construída a partir de um dataset do Kaggle, previamente extraído e refinado dentro da própria codebase em que este notebook está hospedado.

O objetivo é responder, de forma analítica, à pergunta: **“O quão bom é Max Verstappen?”**
Para isso, sigo uma narrativa dedutiva, partindo de indicadores mais gerais — como número de vitórias, pódios e títulos — a fim de estabelecer um panorama inicial e nivelar o contexto para qualquer leitor: estamos, sem dúvida, diante de um dos maiores pilotos da Fórmula 1 moderna.

Em seguida, à medida que a análise se aprofunda, passo a examinar métricas mais específicas e comparativas — como consistência de desempenho, diferença em relação aos companheiros de equipe e outros fatores que ajudam a isolar variáveis externas, como o desempenho do carro ou da equipe.
O objetivo é ir além da “obviedade” dos números brutos, buscando entender até que ponto o domínio de Verstappen é reflexo do carro ou resultado do próprio piloto.

#### Avisos gerais em relação a análise:

- Os dados utilizados foram extraídos do dataset Kaggle: https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020
    - Ver notebook: `explore_data.ipynb` para ter acesso ao "check de sanidade" que fiz do dataset e entender um pouco de suas limitações.
- No momento da análise, o campeonado de 2025 segue em andamento e dados referentes ao mesmo não estarão presentes na análise.

## Setup da análise:

Imports, configs etc...

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import utils as fxns
import os
from pathlib import Path
import utils as fxns
from dateutil.relativedelta import relativedelta

# Caminho absoluto da raiz do projeto
project_root = Path.cwd().parent  # se o notebook está em /notebooks
os.chdir(project_root)

print("Diretório atual:", Path.cwd())

In [ ]:
from src.modules.db_reader import DbReader
from src.data_viz.plotter import Plotter
f1_db = DbReader()
plotter = Plotter()

### Importando os datasets:

In [ ]:
# Corridas:

df_races = f1_db.run_query_file("data/db_queries/race_results_report.sql").drop_duplicates()
df_races['race_date'] = pd.to_datetime(df_races['race_date'])

df_races

In [ ]:
# Infos sobre pilotos:

df_drivers = f1_db.run_query_file("data/db_queries/drivers.sql").drop_duplicates()
df_drivers['dob'] = pd.to_datetime(df_drivers['dob'])
df_drivers


In [ ]:
df_lap_times = f1_db.run_query_file("data/db_queries/lap_times_report.sql").drop_duplicates()
df_lap_times['race_date'] = pd.to_datetime(df_lap_times['race_date'])
df_lap_times

In [ ]:
# Vou adicionar a idade no dataset de corridas, já que vou acabar usando isso agora e em outros momentos:

df_races = pd.merge(
    df_races,
    df_drivers[['driver_id', 'dob']],
    on='driver_id',
    how='left'
)

df_races['driver_age_at_race'] = df_races.apply(lambda row: fxns.calcula_idade(row['dob'], row['race_date']), axis=1)
df_races

## Começando do começo...

Max Verstappen chegou fazendo barulho na F1, sendo o piloto mais jovem da história da F1 ao largar em uma corrida:

In [ ]:
df_first_race = fxns.gerar_dataset_primeiro_evento(df_races, df_drivers)


In [ ]:
df_top_10_jovens_largada = df_first_race.sort_values("idade_primeiro_evento").head(10)
df_top_10_jovens_largada

In [ ]:
fxns.gera_graf_top_10_mais_jovens(df_top_10_jovens_largada, "Os 10 pilotos mais jovens a largar na Fórmula 1", "Idade na estreia (anos)", "Verstappen")

E vai além, ele não apenas possui essa marca considerando estréia, mas pra vitória na F1 também.

In [ ]:
# Vamos gerar um dataset com a primeira vitória de cada piloto por idade no momento em que conquistou a sua vitória:

df_race_wins = df_races[df_races['finishing_position'] == 1].copy()

df_first_win = fxns.gerar_dataset_primeiro_evento(df_race_wins, df_drivers)

In [ ]:
# Agora pego os 10 mais jovens:

df_top_10_jovens_vencedores = df_first_win.sort_values("idade_primeiro_evento").head(10)
df_top_10_jovens_vencedores

In [ ]:
fxns.gera_graf_top_10_mais_jovens(df_top_10_jovens_vencedores, "Os 10 pilotos mais jovens a vencer na Fórmula 1", "Idade na estreia (anos)", "Verstappen")

## Mas... e desde então?

Beleza, acima conseguimos ver que o Verstappen não só foi o mais jovem estreante da F1 mas também foi o mais jovem a ganhar uma corrida, um ano após a sua estreia.

Mas, desde então, se fizermos uma visão geral dos números de vitórias, poles etc, Verstappen continua tendo marcas interessantes? Vamos dar uma olhada em alguns números gerais e a partir disso podemos ir aprofundando as nossas análises.

### Pole Positions:

In [ ]:
df_races

In [ ]:
df_pole_positions = df_races[df_races['starting_position'] == 1].copy()

df_pole_positions_agrup = df_pole_positions.groupby('driver_full_name').agg(
    {
        'race_name': 'count',
        'driver_age_at_race': 'max'
    }
).rename(columns={'race_name': 'pole_positions', 'driver_age_at_race': 'oldest_pole_position_age'}).reset_index().sort_values(by='pole_positions', ascending=False)

df_pole_positions_agrup

In [ ]:
# Vamos tornar isso visual pra ficar mais fácil de expor a informação:

df_poles_top10 = df_pole_positions_agrup.head(10).copy()

df_poles_top10['nome_e_idade'] = df_poles_top10.apply(lambda row: f"{row['driver_full_name']} ({round(row['oldest_pole_position_age'], 2)} anos)", axis=1)

fxns.graf_top10_pilotos(
    df_poles_top10,
    col_nome="nome_e_idade",
    col_valor="pole_positions",
    titulo="Top 10 pilotos em número de poles",
    xlabel="Poles",
    nome_a_destacar="Verstappen",
    orientation='vertical',
    mostrar_chips=False
)


Acima não só podemos ver que o Verstappen é o número 5 em maior quantidade de pole positions na história da F1, como também ele é o mais jovem entre todos os pilotos que estão presentes na lista dos Top 10, sendo aproximadamente 5 anos mais jovem do que o 2o piloto mais jovem que seria o Nico Rosberg.

**Obs**.: a idade apresentada é a idade do piloto ao conseguir sua última pole presente na base de dados, faço assim pra tentar nivelar a comparação, ou seja, considero apenas a idade do piloto enquanto ele estava aumentando sua estatística de pole position.

O piloto que chega mais perto de Verstappen em termos de idade e que possui um volume bem considerável de pole positions seria o Leclerc, que cairia bem no Top 11. Porém, possui 26 pole positions, que equivale a 65% da quantidade de pole positions de Max Verstappen:

In [ ]:
df_poles_ver_lec = df_pole_positions_agrup[df_pole_positions_agrup['driver_full_name'].str.contains('Verstappen|Leclerc', case=False, na=False)].copy()

df_poles_ver_lec['nome_e_idade'] = df_poles_ver_lec.apply(lambda row: f"{row['driver_full_name']} ({round(row['oldest_pole_position_age'], 2)} anos)", axis=1)

fxns.graf_top10_pilotos(
    df_poles_ver_lec,
    col_nome="nome_e_idade",
    col_valor="pole_positions",
    titulo="Pole Positions: Verstappen VS LeClerc",
    xlabel="Poles",
    nome_a_destacar="Verstappen",
    orientation='vertical',
    mostrar_chips=False
)


### Vitórias:

In [ ]:
df_race_wins = df_races[df_races['finishing_position'] == 1].copy()

df_race_wins_agrup = df_race_wins.groupby('driver_full_name').agg(
    {
        'race_name': 'count',
        'driver_age_at_race': 'max'
    }
).rename(columns={'race_name': 'race_wins', 'driver_age_at_race': 'oldest_race_win_age'}).reset_index().sort_values(by='race_wins', ascending=False)

df_race_wins_agrup

In [ ]:
# Vamos tornar isso visual pra ficar mais fácil de expor a informação:

df_race_wins_top_10 = df_race_wins_agrup.head(10).copy()

df_race_wins_top_10['nome_e_idade'] = df_race_wins_top_10.apply(lambda row: f"{row['driver_full_name']} ({round(row['oldest_race_win_age'], 2)} anos)", axis=1)

fxns.graf_top10_pilotos(
    df_race_wins_top_10,
    col_nome="nome_e_idade",
    col_valor="race_wins",
    titulo="Top 10 pilotos em número de vitórias",
    xlabel="Vitórias",
    nome_a_destacar="Verstappen",
    orientation='vertical',
    mostrar_chips=False
)


Bom, eu acho que o gráfico acima nem precisa de muita explicação. Podemos ver que o Verstappen é o 3o piloto com maior número de vitórias da história e é o mais jovem por uma margem de 5 anos na lista, sendo que Vettel é o segundo mais jovem na lista de top 10, com 10 vitórias a menos ao final de 2024, ano no qual Vettel está aposentado e Verstappen continua correndo em 2025.

### E se olharmos em termos percentuais?

In [ ]:
# Posso trazer a quantidade total de corridas de cada piloto pro dataset que usei pra ver as vitórias e calcular um percentual

df_driver_total_races = df_races.groupby('driver_full_name').agg({'race_name':'count'}).rename(columns={'race_name':'total_races'})


df_race_wins_agrup = pd.merge(
    left=df_race_wins_agrup,
    right=df_driver_total_races,
    on='driver_full_name',
    how='left'
)

df_race_wins_agrup['win_percentage'] = df_race_wins_agrup['race_wins'] / df_race_wins_agrup['total_races']

In [ ]:
df_race_wins_agrup

In [ ]:
# Vamos dar uma olhada nos top 10 maiores percentuais de vitórias:

df_top_10_win_percentages = df_race_wins_agrup[
    df_race_wins_agrup['total_races'] > 100 # Vou colocar essa restrição porque tem alguns pilotos que participaram de pouquíssimas corridas no passado e tem vitórias, não faz sentido a comparação nesse caso
].sort_values(by='win_percentage', ascending=False).head(10)

In [ ]:
df_top_10_win_percentages

In [ ]:
df_top_10_win_percentages['nome_e_idade'] = df_top_10_win_percentages.apply(lambda row: f"{row['driver_full_name']} ({round(row['oldest_race_win_age'], 2)} anos)", axis=1)

fxns.graf_top10_pilotos(
    df_top_10_win_percentages,
    col_nome="driver_full_name",
    col_valor="win_percentage",
    titulo="Top 10 pilotos em percentual de vitórias",
    xlabel="Poles",
    nome_a_destacar="Verstappen",
    orientation='vertical',
    mostrar_chips=False,
    valor_format_str="{:.1%}"
)

Ao final da temporada de 2024, Verstappen era o 4o piloto com maior percentual de vitórias da F1, considerando os pilotos que tiveram participação em mais de 5 corridas. Mas valem algumas observaçõe que não são tão óbvias ao observar o gráfico:

- O Verstappen tem muito mais corridas do que qualquer piloto que tem um percentual maior do que o dele;
- O Verstappen está com o maior percentual ao se considerar os pilotos da era "moderna".

# Mas e aí?

Todos esses números apresentados até agora são lindos e maravilhosos. Independente de qualquer contraponto, não há de se discutir que o Verstappen hoje em dia é um piloto mais do que consolidado e está entre os maiores nomes que a F1 já teve em sua história.

Porém... Tudo que trouxe até agora são só grandes números, estou comparando valores absolutos com outros pilotos sem isolar um grande fator: a equipe.

A F1, apesar de certas pessoas esquecerem às vezes, não é uma competição apenas entre pilotos, mas também uma competição de construtores (equipes), que acabam por influenciar muito o resultado individual de cada piloto, e aí a questão é a seguinte: como poderíamos tentar isolar esse fator em uma análise?

É muito difícil, mas acredito que com algumas premissas, consigo prosseguir a análise de forma a tentar fazer isso de duas formas: primeiro posso analisar a consistência de tempo de volta dos pilotos (um piloto bom tem que ser um piloto que consegue entregar resultado nos momentos mais importantes e que tem tempos de volta consistentes), e, além disso, podemos comparar os pilotos diretamente com os companheiros de equipe que tiveram, sendo que ambos possuem o mesmo equipamento (premissa importante).

Então agora, pretendo prosseguir a análise sem aprofundar em grandes números absolutos, mas indo um passo adiante na análise, entrando mais em detalhe nos números...

## Análise de Consistência:

A ideia aqui vai ser tentar fazer uma análise da consistência do piloto ao longo das corridas que participou. E eu não quero fazer isso em torno de uma análise de resultados, mas sim de tempos de volta, porque resultados, novamente, podem ser muito influenciados por externalidades (questões mecânicas, performance da equipe etc), mas, em tese, quem tem mais influência na consistência do tempo de volta, seja ele bom ou ruim, é o piloto. Então vamos lá:

In [ ]:
df_lap_times

Passos para análise de consistência:

- Preciso remover voltas que não seriam interessantes eu considerar do dataset pra analisar apenas tempo de volta válida (não quero voltas lentas de SC, ou que o piloto tenha parado no Box poluindo a análise);
- Depois vou calcular o desvio padrão usando a window de prova e piloto (quero o desvio padrão que cada piloto teve em cada prova);
- Análise da distribuição desse desv. pad. -> A "largura" da distribuição vai nos dizer algo sobre a consistência;
- Análise da média do desvio padrão 0-> quanto maior a média, maior a inconsistência

Obs.: Não sei se vou conseguir fazer uma análise da base toda de uma vez, porque vai ter pilotos com poucas corridas etc que vão atrapalhar a análise. Nesse caso, possivelmente só vai fazer sentido eu comparar pilotos diretamente comparáveis da era atual e nomes maiores que sabemos que são considerados grandes pilotos da F1.

### Fazendo o tratamento do dataset:

#### Primeiro quero tentar tirar as voltas de *Safety Car*: 

Vou tentar fazer isso por meio de uma abordagem "heurística": Eu vou calcular a mediana do tempo de volta de uma corrida e vou comparar com a volta do piloto usando um *threshold*, quando o tempo de volta ultrapassar esse valor (que acontece quando a volta, em tese, é BEM mais lenta que uma volta normal), e vou colocar a condição pra verificar que seja uma vola em que o piloto não perdeu posições (se ele rodou, o tempo de volta vai ser maior mas ele vai ter perdido posições, se for um SC por um motivo externo, ele não deveria perder posições).

Dessa forma (digo isso depois de ter feito essa análise algumas vezes e olhado repetidamente o dataset resultante), acredito que eu tenha conseguido, da melhor forma que pensei, retirar voltas que são lentas por motivos de corrida, mas ao mesmo tempo, manter voltas em que o piloto possa ter cometido algum erro ou então ter sido mais lento por falta de consistência mesmo.

In [ ]:
df_lap_times = fxns.identificar_voltas_safety_car(
    df_laps=df_lap_times,
    threshold_percent=1.20 # Pra eu considerar como volta de SC, o ritmo tem que ter diminuído em pelo menos 20% (chute com critério em breves pesquisas)    
    )

In [ ]:
df_lap_times

In [ ]:
# Vamos fazer um "check de sanidade" nessa flag que eu acabei de criar:

df_check_sc = df_lap_times.groupby(
    by=[
        'race_name',
        'race_date',
        'is_safety_car_lap'
    ]
).agg({
    'lap_time_ms':'mean'
}).reset_index()

In [ ]:
df_check_sc.sort_values('lap_time_ms', ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))

sns.histplot(
    data=df_check_sc,
    x='lap_time_ms',
    hue='is_safety_car_lap',
    bins=1000,
    ax=ax
)

ax.set_title('Distribuição de tempos médios de volta: Volta Normal VS SC')
ax.set_xlabel('Tempo de Volta (ms)')
ax.set_ylabel('Contagem')
ax.legend(title='Flag Safety Car', labels=['Safety Car', 'Volta Normal'])

É difícil fazer uma validação plena da informação acima, mas, vendo o gráfico, conseguimos observar que as voltas classificadas como lentas (Safety Car) estão mais esparsas e mais lentas que as voltas normais, que são maiores em quantidade e mais concentradas, o que vai em linha com o esperado, portanto, considero que a flag está fazendo sentido no momento.

### Removendo voltas a serem desconsideradas na análise:

Na minha visão, para análises de tempo de volta, tenho que tirar:
- Voltas de SC, que foi as que indicamos acima;
- Primeiras voltas, sendo que não são representativas do ritmo de corrida ao longo das demais voltas;
- Voltas em que o piloto parou no pit stop, e isso, conseguimos ter a indicação a partir do dataset de pit stop, sendo que previamente já incluimos uma flag pra identificar esses casos;
- Removo eventos em que o piloto não terminou (geralmente a última volta possui valores bem elevados nesses casos).

In [ ]:
df_lap_times_valid = fxns.filtrar_voltas_para_analise(df=df_lap_times)

In [ ]:
df_lap_times_valid

### Criando uma métrica de consistência

Agora que tenho o meu dataset "limpo", vou tentar calcular a consistência do piloto por meio do desvio padrão do tempo de volta de cada piloto em cada etapa. A ideia aqui é a seguinte:

- Analisar a distribuição do desvio padrão -> Quanto mais agrupada a distribuição em menores valores, significa que o piloto é mais consistente entre as diferentes corridas.
- Analisar a média do desvio padrão -> Quanto menor o o valor, significa que o piloto possui maior consistência entre as diferentes corridas.

A questão (depois de já ter tentado fazer essa análise algumas vezes) é que algumas corridas tem um desvio padrão de tempo de volta muito alto por natureza, e isso acaba atrapalhando bem a análise em certas comparações (corridas em que há mudança de clima possuem um alto desvio padrão natural embutido), então, o que terei que fazer é uma espécie de **"normalização relativa"** dos desvios padrões da seguinte forma:

- Calcularei os desvios padrões de cada piloto em cada evento, e farei a média geral desses valores também, irei então, calcular a diferença entre o desvio padrão do piloto e a média dos desvios padrões entre os pilotos, assim, eu vou beneficiar os pilotos que possuirem maior consistência (menor variação do tempo de volta) contra a variação "inerente" (representada pela média dos desvios padrões).

Vamos lá e vou tentar deixar claro conforme eu for chegando em conclusões:

In [ ]:
df_lap_times_std = (
    df_lap_times_valid.groupby(['race_name', 'year', 'driver_full_name'])['lap_time_ms']
    .agg(['std', 'mean'])
    .rename(columns={'std': 'lap_time_std', 'mean': 'lap_time_mean'})
    .reset_index()
)

df_lap_times_std['lap_time_std_mean'] = df_lap_times_std.groupby(['race_name', 'year'])['lap_time_std'].transform('mean')

df_lap_times_std['lap_time_std_driver_vs_mean'] = df_lap_times_std['lap_time_std'] - df_lap_times_std['lap_time_std_mean']

df_lap_times_std

Antes de qualquer coisa, vamos dar uma olhada na consistência do Verstappen pra eu explicar o que estou tentando analisar:

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))

df_plot = df_lap_times_std[df_lap_times_std['driver_full_name'].isin(['Max Verstappen'])]

# Define uma paleta de cores fixa para garantir consistência
cor_base = "#FF7009"      # Laranja para o piloto base (Verstappen)
palette = {'Max Verstappen': cor_base}

# Calcula o número de corridas
n_corridas_base = len(df_plot[df_plot['driver_full_name'] == 'Max Verstappen'])

# Plot do histograma e da curva de densidade (KDE)
sns.histplot(data=df_plot, x='lap_time_std_driver_vs_mean', bins=250, kde=True, ax=ax, color=cor_base)

# Adiciona linhas verticais para a média de cada piloto
media_base = df_plot[df_plot['driver_full_name'] == 'Max Verstappen']['lap_time_std_driver_vs_mean'].mean()

ax.axvline(media_base, color=cor_base, linestyle='--', linewidth=2, label=f'Média {'Max Verstappen'.split(" ")[-1]}: {media_base:.2f}')

# Títulos e rótulos
ax.set_title(f"Análise de Consistência: {'Max Verstappen'}", fontsize=16)
ax.set_xlabel(f"{'lap_time_std_driver_vs_mean'} (Menor é mais consistente)", fontsize=12)
ax.set_ylabel("Frequência (nº de corridas)", fontsize=12)

# Atualiza a legenda para incluir a contagem de corridas
handles, _ = ax.get_legend_handles_labels()
labels = [f'{'Max Verstappen'} ({n_corridas_base} corridas)'] + [h.get_label() for h in handles[2:]]
ax.legend(handles=handles, labels=labels)

ax.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

#### Vou explicar o que eu estou traznedo aqui acima:

Neste gráfico, visualizamos a distribuição do 'lap_time_std_driver_vs_mean' para Max Verstappen. Essa métrica representa a diferença entre o desvio padrão de Max Verstappen e o desvio padrão médio de todos os pilotos em determinado evento.

A linha vertical tracejada indica a média dos valores de 'lap_time_std_driver_vs_mean' de Verstappen. Uma média menor que 0, que é o caso de Verstappen, significa que ele tende a ser mais consistente que a média dos pilotos.

Ou seja, amostras à esquerda dessa linha representam corridas em que o piloto foi mais consistente que a média, e os valores à direita indicam corridas em que a consistência foi abaixo da média (desvio padrão maior que a média). O ideal é observar uma concentração maior de corridas à esquerda, sinalizando que, na maior parte do tempo, o piloto manteve um desempenho consistente, em que seus tempos de volta variaram menos que a média. 

**Importante**: os dados que estou olhando no momento não refletem TEMPO DE VOLTA, mas sim são um reflexo de CONSISTÊNCIA. Ou seja, valores a esquerda não significam voltas mais rápidas, mas sim que os tempos de volta oscilaram menos do que de outros pilotos. Vou olhar performance, tempos de volta de uma outra forma logo depois...

### Comparando com outros pilotos

Agora que expliquei o que estou tentando fazer aqui, eu posso fazer comparação entre a distribuição que vimos acima de Verstappen, com uma média menor do que 0 e com uma distribuição aparentemente concentrada em torno do 0, com demais pilotos, vendo como as coisas se comparam.

O que queremos ver aqui decorre do que expliquei acima:

- Menor média = maior consistência
- Distribuição concentrada = pouca variação do tempo de volta
- Amostras presentes mais à esquerda = consistência maior que a média
- Amostras à direita = pior consistência do que a média
- Cauda alongada à direita = possíveis amostras nas quais o piloto cometeu algum erro e perdeu posições

etc...

In [ ]:
pilotos_para_comparar = [
    # Campeões e Rivais Históricos
    'Lewis Hamilton',
    'Sebastian Vettel',
    'Fernando Alonso',
    'Nico Rosberg',
    'Kimi Räikkönen',
    
    # Companheiros de Equipe (atuais e passados)
    'Sergio Pérez',
    'Daniel Ricciardo',
    'Alexander Albon',
    'Pierre Gasly',
    'Carlos Sainz',
    
    # Nova Geração e Rivais Atuais
    'Charles Leclerc',
    'Lando Norris',
    'George Russell',
    'Oscar Piastri',
    'Valtteri Bottas'
]

fxns.comparar_consistencia_pilotos_hist(
    df_consistencia=df_lap_times_std,
    pilotos_a_comparar=pilotos_para_comparar,
    metrica='lap_time_std_driver_vs_mean',
    bins=250
)


Bom, eu acredito que cheguei em alguma análise com um bom critério, porque os resultados nas comparações acima foram "consistentes" (risos):

- O Verstappen parece possuir uma consistência fora do normal, olhando cada gráfico acima, a média de seu desvio padrão relativo é menor do que todos os outros pilotos que estamos trazendo na comparação.
- O único piloto que chega mais perto (tendo uma quantidade muito alta de corridas também, o que é mais um reforço em termos de consistência) é ninguém mais, ninguém menos que Lewis Hamilton (que surpresa)...

Para tentar simplificar essa visão de comparação de médias, podemos ir direto nisso também:



In [ ]:
df_lap_times_std

In [ ]:
df_lap_times_std_agrup = df_lap_times_std[
    # Quero só trazer uma era mais "moderna", por exemplo, não vejo sentido em comparar com Senna, por n motivos
    (df_lap_times_std['year']>2010) &\
    (df_lap_times_std['driver_full_name'].isin(pilotos_para_comparar + ['Max Verstappen', 'Lance Stroll']))
    ].groupby( 
    by=[
        'driver_full_name',
    ]
).agg(
    {
        'lap_time_std_driver_vs_mean':'mean'
    }
).reset_index()

df_lap_times_std_agrup.sort_values('lap_time_std_driver_vs_mean', ascending=True)

In [ ]:
# Define a cor laranja para o Verstappen
cor_verstappen = "#FF7009"

# Cria a paleta de cores.  Outros pilotos serão azuis.
palette = {piloto: cor_verstappen if piloto == 'Max Verstappen' else 'steelblue' for piloto in df_lap_times_std_agrup['driver_full_name']}

# Ordena o DataFrame para que o piloto mais consistente apareça no topo
df_lap_times_std_agrup = df_lap_times_std_agrup.sort_values('lap_time_std_driver_vs_mean', ascending=False)

# Cria o gráfico
fig, ax = plt.subplots(figsize=(12, 8))
sns.barplot(x='lap_time_std_driver_vs_mean', y='driver_full_name', data=df_lap_times_std_agrup, palette=palette, ax=ax)

# Adiciona rótulos e título
ax.set_xlabel('Média lap_time_std_driver_vs_mean (Valores mais negativos indicam maior consistência)')
ax.set_ylabel('Piloto')
ax.set_title('Comparação da Consistência Média por Piloto (Valores mais negativos = Mais consistente)')

# Adiciona grid para facilitar a leitura
ax.grid(axis='x', linestyle='--')

# Remove as bordas do gráfico
sns.despine(left=True, bottom=True)

# Mostra o gráfico
plt.show()

Max Verstappen se apresenta como o piloto mais consistente da análise:

- Seguido de Oscar Piastri, o que é bem interessante, mas uma coisa a se levar em consideração é que o Piastri tem muito menos corridas do que o Verstappen (apenas 38 até onde meu dataset vai), sendo que espera-se ter algo mais representativo ao longo de um número maior de corridas;
- Lewis Hamilton, que possui inclusive mais corridas do que o MV, é o 3o colocado na análise de consistência, o que é consistente com o calibre de pilodo que é, e faz sentido;
- Depois do LH, vemos outros grandes nomes da F1, muitos que já foram campeões, todos se apresentando como mais consistentes do que a média.